# NYC Taxi data

## Import cuxfilter

In [1]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import feature_and_double_base

import cudf

In [2]:
#update data_dir if you have downloaded datasets elsewhere
DATA_DIR = './data'

## Download required datasets

In [ ]:
! curl https://rapidsai-data.s3.us-east-2.amazonaws.com/viz-data/nyc_taxi.tar.gz --create-dirs -o $DATA_DIR/nyc_taxi.tar.gz

In [ ]:
from cuxfilter.sampledata import datasets_check
datasets_check('nyc_taxi', base_dir=DATA_DIR)

## preprocess the data

In [3]:
cudf_df = cudf.read_csv(DATA_DIR + '/nyc_taxi.csv').dropna()

from pyproj import Proj, Transformer

# Apply transformation
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
cudf_df['dropoff_x'], cudf_df['dropoff_y'] = transform_4326_to_3857.transform(
                                                cudf_df['dropoff_latitude'].values_host, cudf_df['dropoff_longitude'].values_host
                                            )

cudf_df = cudf_df.drop(['dropoff_latitude', 'dropoff_longitude'], axis=1)
cudf_df = cudf_df.dropna(axis=0)

# Filter over Manhattan
cudf_df = cudf_df[
    (cudf_df.dropoff_x > -8239910.23) & (cudf_df.dropoff_x < -8229529.24) & (cudf_df.dropoff_y > 4968481.34) & (cudf_df.dropoff_y < 4983152.92)
]

cudf_df.tpep_pickup_datetime = cudf.to_datetime(cudf_df.tpep_pickup_datetime, format="%Y-%m-%d")
cudf_df.tpep_dropoff_datetime = cudf.to_datetime(cudf_df.tpep_dropoff_datetime, format="%Y-%m-%d")
cudf_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dropoff_x,dropoff_y
0,2,2015-01-15,2015-01-15,1,1.59,-73.993896,40.750111,1,N,1,12.0,1.0,0.5,3.25,0.00,0.3,17.05,-8.234835e+06,4.975627e+06
1,1,2015-01-10,2015-01-10,1,3.30,-74.001648,40.724243,1,N,1,14.5,0.5,0.5,2.00,0.00,0.3,17.80,-8.237021e+06,4.976875e+06
3,1,2015-01-10,2015-01-10,1,0.50,-74.009087,40.713818,1,N,2,3.5,0.5,0.5,0.00,0.00,0.3,4.80,-8.238124e+06,4.971127e+06
4,1,2015-01-10,2015-01-10,1,3.00,-73.971176,40.762428,1,N,2,15.0,0.5,0.5,0.00,0.00,0.3,16.30,-8.238108e+06,4.974457e+06
5,1,2015-01-10,2015-01-10,1,9.00,-73.874374,40.774048,1,N,1,27.0,0.5,0.5,6.70,5.33,0.3,40.33,-8.236193e+06,4.976740e+06


## Read the dataset

In [13]:
cux_df = cuxfilter.DataFrame.from_dataframe(cudf_df)

## Define charts

> Uncomment the below lines and replace MAPBOX_TOKEN with mapbox token string if you want to use mapbox map-tiles. Can be created for free here -https://www.mapbox.com/help/define-access-token/

In [14]:
#from cuxfilter.assets.custom_tiles import get_provider, Vendors
#tile_provider = get_provider(Vendors.MAPBOX_LIGHT, access_token=MAPBOX_TOKEN)

In [15]:
chart1 = cuxfilter.charts.scatter(x='dropoff_x',
                                 y='dropoff_y',
                                 aggregate_fn='mean',aggregate_col='payment_type', pixel_shade_type='log', legend_position='top_right',
                                 tile_provider="CartoLight", x_range=(-8239910.23,-8229529.24), y_range=(4968481.34,4983152.92))

chart2 = cuxfilter.charts.bar('passenger_count', data_points=9)
chart3 = cuxfilter.charts.bar('tpep_pickup_datetime')
chart4 = cuxfilter.charts.date_range_slider('tpep_dropoff_datetime')

## Create a dashboard object

In [16]:
d = cux_df.dashboard([chart1, chart2, chart3], sidebar=[chart4], layout=feature_and_double_base, theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASET')

In [17]:
#execute below line for dashboard preview
# await d.preview()

In [18]:
d.show()

Dashboard running at port 40171


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

## Starting the dashboard

1. d.show('current_notebook_url:current_notebook_port') remote dashboard

2. d.app() inline within the notebook cell

Incase you need to stop the server:

- d.stop()

In [ ]:
# d.show() # notebook_url="http://localhost:8888"
# d.app() # run the dashboard within the notebook cell

## Export the queried data into a dataframe

In [ ]:
queried_df = d.export()